In [1]:
# --- Allgemeine Python-Bibliotheken ---
# Diese Bibliotheken bieten grundlegende Funktionen für die Arbeit mit Datenstrukturen, 
# Dateioperationen, Typisierung und mehr. 
import json  
import os  
from typing import TypedDict, Annotated, Sequence, List, Optional  
import operator 

# --- Langchain: Für die Interaktion mit Sprachmodellen und Verarbeitungsketten --- 
# Langchain ist eine Bibliothek, die die Verwendung von Large Language Models (LLMs) 
# vereinfacht. Sie bietet Tools zur Kommunikation mit LLMs, zur Erstellung von Prompts, 
# zur Verkettung von LLM-Aufrufen und zum Parsen der Ausgaben.
from langchain_openai import ChatOpenAI  
from langchain_core.messages import BaseMessage, HumanMessage, SystemMessage  
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder, PromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate  
from langchain_core.output_parsers import PydanticOutputParser, JsonOutputParser  

# --- Pydantic: Für Datenvalidierung und -modellierung ---
# Pydantic ist eine Bibliothek, die die Definition von Datenmodellen mit Typsicherheit ermöglicht. 
# Sie wird hier verwendet, um die vom Sprachmodell generierte Ausgabe zu strukturieren und zu validieren.
from pydantic import BaseModel, Field

# --- Langgraph: Für die Workflow-Definition und -Ausführung ---
# Langgraph ermöglicht die Definition komplexer Workflows, die aus verschiedenen Schritten bestehen. 
# In diesem Projekt wird Langgraph verwendet, um den Prozess der Prozessmodellierung mithilfe des 
# Sprachmodells zu orchestrieren.
from langgraph.graph import StateGraph, END 

# --- .env-Datei laden ---
# Diese Zeile ermöglicht das Laden von Umgebungsvariablen (z.B. API-Keys) 
# aus einer .env-Datei. Dies ist nützlich, um sensible Daten nicht direkt im Code zu speichern.
from dotenv import load_dotenv, find_dotenv

In [2]:
# Setup um OpenAI LLM zu verwenden
# Wenn ein Open Source LLM verwendet werden soll, dann diesen Code-Abschnitt nicht ausführen!
OPENAI_API_KEY="Hier OpenAI-API-Key eintragen"

In [3]:
# Lade Umgebungsvariablen aus der .env-Datei
load_dotenv(find_dotenv())

# Setze den API-Key für Langchain aus der Umgebungsvariable.
os.environ["LANGCHAIN_API_KEY"] = str(os.getenv("LANGCHAIN_API_KEY"))

# OpenAI API-Key für den Zugriff auf das LLM
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

# LangSmith verwenden, um die einzelnen Runs der Methode genau nachverfolgen zu können
# Aktiviere detaillierte Protokollierung für Langchain
# LangSmith API-Endpunkt
# LangSmith Projektname
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"] = "Evaluation HumanInTheLoop"

In [4]:
# Initialisiere das LLM GPT-4 mit spezifischen Parametern.
# temperature=0: Steuert die Zufälligkeit der Ausgabe. Niedrigere Werte führen zu deterministischeren Ausgaben.
# model_name="gpt-4o": Spezifiziert das zu verwendende LLM.
# max_tokens=None: Keine Beschränkung der maximalen Tokenanzahl in der Ausgabe.

llm = ChatOpenAI(temperature=0, model_name="gpt-4o", max_tokens=None)

In [5]:
# Definiere einen benutzerdefinierten Dictionary-Typ 'AgentState', um verschiedene Nachrichtensequenzen zu speichern.
# Dieser Typ wird verwendet, um den Zustand des Agenten während des Workflows zu verfolgen.
# 
# 'messages': Enthält alle ausgetauschten Nachrichten zwischen User und LLM.
# 'process_description': Enthält die Beschreibung des Prozesses, die schrittweise verfeinert wird.
# 'process_modell': Speichert das generierte Petri-Netz-Modell des Prozesses.
# 'critique': Beinhaltet die vom Modell generierte Kritik am Petri-Netz.
class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]
    process_description: Annotated[Sequence[BaseMessage], operator.add]
    process_modell: Annotated[Sequence[BaseMessage], operator.add]
    critique: Annotated[Sequence[BaseMessage], operator.add]

In [6]:
# Generiert Klärfragen basierend auf der Prozessbeschreibung.
# Diese Funktion analysiert die vom Benutzer bereitgestellte Prozessbeschreibung 
# und formuliert mithilfe eines Large Language Models (LLM) gezielte Fragen, 
# um Unklarheiten oder fehlende Informationen zu identifizieren. 

# Args:
    # state (AgentState): Der aktuelle Status des AgentState, enthält die Prozessbeschreibung.

# Returns:
    # dict: Ein Dictionary, das die generierten Fragen im AgentState unter dem Schlüssel "messages" speichert.         
def generateQuestions(state):

    # Extrahiere die Prozessbeschreibung aus dem AgentState.
    messages = state["process_description"]
    process_description = messages[0]
    
    # Definiere den Prompt für die Generierung von Fragen.
    # Der Prompt enthält detaillierte Anweisungen für das LLM, 
    # welche Arten von Fragen relevant sind und wie diese formuliert sein sollten.
    # Zusätzlich werden Beispiele für gute und schlechte Fragen gegeben.
    question_prompt = """
    You are an expert in process modeling and Petri Nets. Your task is to formulate questions based on a provided process description. The goal is to clarify any ambiguities that may affect the accurate modeling of the process as a Petri Net. Your questions should focus on critical aspects where the process description is unclear or incomplete, addressing the following areas:

    1. Process Sequence:
    - Identify the key activities and their order.
    - Clarify the start and end points of the process.

    2. Parallelism and Concurrency:
    - Determine which activities can occur simultaneously.
    - Identify dependencies between concurrent activities.

    3. Decisions and Choices:
    - Clarify where decision points occur in the process.
    - Identify the branches that result from these decisions.

    4. Loops and Cycles:
    - Identify any repetitive activities and determine where loops or cycles might occur in the process.

    When formulating questions, focus on these principles:
    1. Prioritize Ambiguities: Only ask questions where the process description is vague, unclear, or could lead to multiple interpretations.
    2. Consolidate Questions: Combine related questions to reduce redundancy.
    3. Target Critical Details: Focus on details that directly impact the accuracy of the Petri Net model without delving into unnecessary specifics like decision criteria.

    ### Example Process Description:
    "The manufacturing process begins with assembling components, which may involve various sub-assemblies. After assembly, the product undergoes quality checks, including both visual inspections and functionality tests. Once the product passes these checks, it is packaged and prepared for shipment. Packaging can involve multiple steps depending on the product type, and the process concludes with a final review before shipment."

    ### Example Targeted Questions:

    **Process Sequence:**
    1. "The description mentions assembling components but does not specify the order of sub-assemblies. What is the sequence of sub-assemblies in this process?"
    2. "What marks the end of the packaging phase, and what is the final activity before the product is shipped?"

    **Parallelism and Concurrency:**
    3. "During the quality checks, can visual inspections and functionality tests be performed in parallel, or must they be sequential?"
    4. "Are the steps in the packaging phase independent, or do they need to be performed in a specific order?"

    **Decisions and Choices:**
    5. "Is there a decision point after the quality checks? If so, what are the possible paths the process could take?"

    **Loops and Cycles:**
    6. "If the product fails a quality check, does the process loop back to a previous phase, such as assembly, or is there a different course of action?"

    ### Example of Non-Targeted Questions to Avoid:

    **Process Sequence:**
    1. "What are the steps in the process?" (Too broad and not focused on specific ambiguities.)
    2. "What happens after the process starts?" (Vague and non-specific.)

    **Parallelism and Concurrency:**
    3. "Can tasks happen simultaneously?" (Overly general and not focused on specific activities.)
    4. "Are any activities parallel?" (Too broad without identifying specific tasks.)

    **Decisions and Choices:**
    5. "Is there a decision point in the process?" (Lacks focus on where or why decisions occur.)
    6. "What choices exist in the process?" (Non-specific and doesn’t target particular ambiguities.)

    **Loops and Cycles:**
    7. "Are there any loops in the process?" (Too general and doesn’t specify potential repetitive activities.)
    8. "Does anything repeat in the process?" (Vague and lacks focus on specific cycles or loops.)

    Focus your questions on the most critical ambiguities or gaps in the process description to ensure an accurate Petri Net model with minimal but essential inquiries.
    """
    
    # Erstelle die Eingabe für das LLM.
    # Die Eingabe besteht aus einer Systemnachricht (Prompt) und einer Benutzernachricht 
    # (Prozessbeschreibung).
    message = [
    SystemMessage(content=question_prompt),
    HumanMessage(content=process_description)]
    
    # Rufe das LLM auf und speichere die generierten Fragen.
    questions = llm.invoke(message)
    
    # Speichere die generierten Fragen im AgentState 'messages'.
    return {"messages": [questions]}

# Fragt den Benutzer nach Antworten auf die generierten Fragen.
# Diese Funktion präsentiert dem Benutzer die vom LLM generierten Fragen und 
# nimmt seine Antworten entgegen. Die Fragen und Antworten werden paarweise 
# gespeichert, um später in die Prozessbeschreibung integriert zu werden.

# Args:
    # state (AgentState): Der aktuelle Status des Agenten, enthält die generierten Fragen.

# Returns:
    # dict: Ein Dictionary, das die Fragen-Antwort-Paare im AgentState unter dem Schlüssel "messages" speichert.
def ask_human(state):
    
    # Extrahiere die generierten Fragen aus dem AgentState.
    # Verwende das aktuellste Element der 'messages'-Liste 
    # (die aktuellsten generierten Fragen).
    messages = state["messages"]
    questions = messages[-1]
  
    # Zeige die generierten Fragen dem Benutzer an.
    print(questions)
    
    # Warte auf die Antwort des Benutzers.
    response = input("Your response to the questions: ")
    
    # Verknüpfe die Fragen mit den Antworten des Benutzers.
    if response == "n":
        raise ValueError
    
    # Gib die Fragen und Antworten im AgentState zurück.
    message_pairs = [(questions, response)]
    
    # Speichere die Fragen und Antworten im AgentState unter 'messages'
    return {"messages": [message_pairs]}

# Integriert die Antworten des Benutzers in die Prozessbeschreibung.
# Diese Funktion nutzt ein LLM, um die vom Benutzer gegebenen Antworten auf die 
# Klärfragen zu analysieren und die ursprüngliche Prozessbeschreibung 
# entsprechend zu erweitern. 

# Args:
    # state (AgentState): Der aktuelle Status des Agenten, enthält die Prozessbeschreibung und die Fragen-Antwort-Paare. 

# Returns:
    # dict: Ein Dictionary, das die erweiterte Prozessbeschreibung im AgentState unter dem Schlüssel "process_description" speichert.
def incorporate(state):
    
    # Definiere den Prompt für die Integration der Antworten in die Prozessbeschreibung.
    # Der Prompt enthält Anweisungen für das LLM, wie die Antworten zu verarbeiten sind und 
    # welche Struktur die Ausgabe haben soll.
    incorporate_prompt = """
    You are an expert in process modeling and Petri Nets. Your task is to enhance a given process description by addressing specific questions. This step aims to provide additional clarity and detail to the original process description by listing the new information separately under "Additional Information" rather than integrating it directly into the process description.

    ### Input:
    - **Original Process Description:**
    A textual description of the process.

    - **Questions:**
    A set of questions aimed at clarifying and detailing the process.

    - **Answers:**
    Corresponding answers to the questions that provide additional information about the process.

    ### Steps to Enhance the Process Description:

    1. **Review Original Process Description:**
    - Understand the existing sequence of activities and flow.

    2. **Analyze Questions and Answers:**
    - Identify key details and clarifications provided by the answers.

    3. **Document Additional Information:**
    - Instead of integrating the additional information directly into the original process description, list it separately under "Additional Information."

    4. **Ensure Clarity and Consistency:**
    - Ensure that the original process description remains clear, detailed, and logically consistent without the direct integration of the new information.

    ### Example Input:

    - **Original Process Description:**
    "A customer places an order online. The order is received and processed by the staff. If the item is in stock, it is packaged and shipped. If the item is out of stock, the customer is notified, and the order is cancelled or put on backorder."

    - **Questions:**
    1. What is the first step in the process?
    2. Can the order be processed and checked for stock simultaneously?
    3. Is there a decision point if the item is out of stock?
    4. What happens if the item is out of stock?

    - **Answers:**
    1. The first step is the customer visiting the online store.
    2. Yes, the order processing and stock checking can occur simultaneously.
    3. Yes, if the item is out of stock, the customer must decide whether to cancel the order or place it on backorder.
    4. The customer is notified, and they can choose to cancel the order or place it on backorder.

    ### Example Output:

    - **Enhanced Process Description:**
    "A customer places an order online. The order is received and processed by the staff. If the item is in stock, it is packaged and shipped. If the item is out of stock, the customer is notified, and the order is cancelled or put on backorder."

    - **Additional Information:**
    1. The process begins with the customer visiting the online store.
    2. Order processing and stock checking can occur simultaneously.
    3. If the item is out of stock, there is a decision point where the customer chooses whether to cancel the order or place it on backorder.

    ### Notes:
    - Ensure that the enhanced process description logically integrates the new information without altering the original text.
    - Maintain clarity and coherence in the original process description.
    - List any additional details separately under "Additional Information" to provide a complete and accurate depiction of the process without integrating new information directly into the process flow.
    """
    
    # Extrahiere notwendige Informationen aus dem AgentState.
    messages = state["messages"]
    process = state["process_description"]
    
    process_description = process[-1]
    message_pairs = messages[-1]
    
    # Wandle die Prozessbeschreibung und die Fragen-Antworten-Paare in Strings um.
    process_description_str = str(process_description)
    
    # Wandle die Prozessbeschreibung und die Fragen-Antworten-Paare in Strings um.
    message_pairs_str = '\n'.join([f"Question: {pair[0]}\nAnswer: {pair[1]}" for pair in message_pairs])
    process_description_message_pairs = f"{process_description_str}\n{message_pairs_str}"
    
    # Erstelle die Eingabe für das LLM.
    message = [
    SystemMessage(content=incorporate_prompt),
    HumanMessage(content=process_description_message_pairs)]
    
    # Rufe das LLM auf und speichere die erweiterte Prozessbeschreibung.
    response = llm.invoke(message).content
    
    # Speichere die erweiterte Prozessbeschreibung im AgentState unter 'process_description'.
    return {"process_description": [response]}  # Ensure it's a list


# Strukturiert die Prozessbeschreibung für die Petri-Netz-Modellierung.
# Diese Funktion verwendet ein LLM, um die erweiterte Prozessbeschreibung 
# in eine strukturierte Form zu bringen, die die spätere Modellierung 
# als Petri-Netz erleichtert.  

# Args:
    # state (AgentState): Der aktuelle Status des Agenten, enthält die Prozessbeschreibung. 

# Returns:
    # dict: Ein Dictionary, das die strukturierte Prozessbeschreibung im AgentState unter dem Schlüssel "process_description" speichert.
def structure(state):
    
    # Definiere den Prompt für die Strukturierung der Prozessbeschreibung.
    # Der Prompt enthält detaillierte Anweisungen, wie die Prozessbeschreibung zu strukturieren ist, 
    # um die spätere Modellierung als Petri-Netz zu erleichtern.
    structure_prompt = """
    <prompt>
        <description>You are an expert in process modeling and Petri Nets. Your task is to structure a detailed process description in a way that facilitates its transformation into a Petri Net model. The structured description should follow the exact order of activities, decision points, parallel processes, loops, and endpoints as described in the original process. When a loop or cycle is present, ensure that the entire loop, including any decisions that lead back to the start of the loop, is described within the "Loops and Cycles" section, avoiding unnecessary subdivisions.</description>

        <Purpose>
            The goal is to accurately reflect the process flow by structuring the description in a clear and logical sequence. Each section should be maintained as a continuous block until a different type of section naturally occurs in the process. Loops and cycles should be fully contained within their section, including any related decisions, to ensure the model can be easily converted into a Petri Net without unnecessary subdivision.
        </Purpose>

        <Modeling_Guidelines>
            1. **Understand the Process:**
                - Carefully read the process description to understand the overall goal and sequence of the process.
                - Identify key activities, decisions, parallel processes, loops, and endpoints that define the workflow.

            2. **Group Related Activities Together:**
                - Group all activities that belong to the same logical phase into a single **Sequence of Activities** section.
                - Only transition to a new section type (e.g., **Decision Points**, **Parallel Activities**, **Loops and Cycles**) when there is a clear shift in the type of process element.

            3. **Avoid Unnecessary Subdivisions:**
                - Do not break a **Sequence of Activities** into multiple sections unless there is an explicit change in the process type.
                - When a loop or cycle is identified, include the entire loop, including any decision points that control the loop, within the **Loops and Cycles** section.

            4. **Identify and Respect Section Boundaries:**
                - Clearly mark where one section ends and another begins, but only when a different type of process element is introduced.
                - Ensure that all decisions that lead back to the start of a loop are incorporated within the **Loops and Cycles** section, and do not create a separate **Decision Points** section for them.

        </Modeling_Guidelines>

        <Examples>
            **Example 1**:
            ### Structured Process Description:

            #### Start Point:
            - **Questionnaire Process Initiation:**
            - The process begins with the customer office sending a questionnaire to the customer by e-mail.

            #### Sequence of Activities:
            1. **Send Questionnaire:**
            - The customer office sends the questionnaire to the customer by e-mail.
            2. **Customer Fills Out Questionnaire:**
            - The customer fills out the questionnaire.
            3. **Customer Sends Back Questionnaire:**
            - The customer sends the completed questionnaire back to the customer office.
            4. **Check Questionnaire:**
            - Upon receipt, the questionnaire is carefully checked in the office.

            #### Loops and Cycles:
            - **Questionnaire Completion Loop:**
            - The process may loop back to the initial step of sending the questionnaire if any information is missing.
            - **Condition to continue the loop:**
                - The office checks if the questionnaire is complete.
                - **If Complete:**
                - The complete questionnaire is entered into the system, and the process is finalized.
                - **If Incomplete:**
                - The email is sent to the customer again, and the loop starts over from the beginning.

            #### End Points:
            - **Process Finalized:**
            - The process ends when the complete questionnaire is entered into the system and the process is finalized.

            **Example 2**:
            ### Structured Process Description:

            #### Start Point:
            - **Initiation of Marketing Campaign:**
            - The process begins with the initiation of a new marketing campaign.

            #### Sequence of Activities:
            1. **Market Research:**
            - Research is conducted to understand the target audience and market trends.
            2. **Campaign Strategy Development:**
            - A strategy is developed based on the research findings.
            3. **Content Creation:**
            - Marketing content is created in line with the campaign strategy.
            4. **Campaign Launch:**
            - The campaign is launched across selected marketing channels.
            5. **Performance Monitoring:**
            - The performance of the campaign is monitored in real-time.

            #### Loops and Cycles:
            - **Campaign Adjustment Loop:**
            - The process may loop back to the strategy development phase if the campaign is not performing as expected.
            - **Condition to continue the loop:**
                - The campaign performance is reviewed.
                - **If Successful:**
                - The campaign continues without adjustment.
                - **If Unsuccessful:**
                - The strategy is adjusted, and the campaign is relaunched.

            #### End Points:
            - **Campaign Concluded:**
            - The process ends when the marketing campaign concludes successfully.

        </Examples>
    </prompt>
    """
    
    # Extrahiere die Prozessbeschreibung aus dem AgentState.
    process = state["process_description"]
    process_description = process[-1]
    
    # Erstelle die Eingabe für das LLM.
    message = [
    SystemMessage(content=structure_prompt),
    HumanMessage(content=process_description)]
    
    # Rufe das LLM auf und speichere die strukturierte Prozessbeschreibung.
    response = llm.invoke(message).content
    
    # Speichere die strukturierte Prozessbeschreibung im AgentState unter 'process_description'.
    return {"process_description": [response]}

# Modelliert die Prozessbeschreibung als Petri-Netz.
# Diese Funktion nimmt den aktuellen Agentenstatus entgegen, extrahiert 
# die strukturierte Prozessbeschreibung und wandelt sie schrittweise 
# in ein Petri-Netz-Modell um. 

# Args:
    # state (AgentState): Der aktuelle Status des Agenten, enthält die Prozessbeschreibung.

# Returns:
    # dict: Ein Dictionary, das das generierte Petri-Netz-Modell unter dem Schlüssel "process_modell" enthält.
def modell(state):
    
    # Definiere den Prompt für die Modellierung der Prozessbeschreibung als Petri-Netz.
    # Der Prompt enthält detaillierte Anweisungen, wie die einzelnen Elemente der 
    # Prozessbeschreibung (Startpunkt, Sequenzen, Entscheidungen, etc.) in 
    # Petri-Netz-Elemente (Stellen, Transitionen, Kanten) zu überführen sind.
    modell_prompt = """
    <prompt>
        <description>You are an AI model specialized in creating Petri nets from structured process descriptions. A process description will be provided in a clearly defined format. Your task is to analyze each section of the process description and create a Petri net based on the given modeling instructions.</description>
        
        <constructs>
            <generalDescription>General Description of Constructs</generalDescription>

            <placesAndTransitions>
                <place>
                    <description>Represents a state or condition in the process.</description>
                    <example>Participated in exam (state)</example>
                    <petrinetConstruct>Place (P)</petrinetConstruct>
                </place>

                <activity>
                    <description>Represents an action or event in the process.</description>
                    <example>Writing an exam (action)</example>
                    <petrinetConstruct>Transition (T)</petrinetConstruct>
                </activity>

                <important>There should never be an edge between two transitions or two places. An edge must always go from a transition to a place or from a place to a transition.</important>
            </placesAndTransitions>
        </constructs>

        <modelingInstructions>
            <section>
                <name>Start Point</name>
                <description>The initial state of the process.</description>
                <petrinetConstruct>A place with no incoming edges, only outgoing edges.</petrinetConstruct>
                <example>
                    <correct>
                        <description>Start Point: Customer brings in a defective computer</description>
                        <place>P1: Customer Brings Computer</place>
                    </correct>
                </example>
            </section>

            <section>
                <name>Sequence of Activities</name>
                <petrinetConstruct>A series of places and transitions connected by edges.</petrinetConstruct>
                <example>
                    <correct>
                        <place>P1: Customer Brings in Defective Computer</place>
                        <edge>from P1 to T1</edge>
                        <transition>T1: Receive Defective Computer</transition>
                        <edge>from T1 to P2</edge>
                        <place>P2: Defective Computer Received</place>
                    </correct>
                </example>
            </section>

            <section>
                <name>Decision Points</name>
                <description>A point where a decision leads to different paths. Just one of the paths can be taken.</description>
                <petrinetConstruct>Place leading to multiple transitions.</petrinetConstruct>
                <example>
                    <correct>
                        <place>P1: Customer Decision on Repair Costs</place>
                        <edge>from P1 to T2</edge>
                        <transition>T2: Accept Costs</transition>
                        <edge>from T2 to P3</edge>
                        <place>P3: Accept Costs</place>
                        <edge>from P1 to T3</edge>
                        <transition>T3: Reject Costs</transition>
                        <edge>from T3 to P4</edge>
                        <place>P4: Reject Costs</place>
                    </correct>
                </example>
            </section>

            <section>
                <name>Parallel Activities</name>
                <description>Activities occurring simultaneously and independently.</description>
                <petrinetConstruct>Transition leading to multiple parallel sequences of places and transitions.</petrinetConstruct>
                <example>
                    <correct>
                        <transition>T3: Begin Parallel Activities</transition>
                        <edge>from T3 to P6 and P9</edge>
                        <place>P6: Hardware Check</place>
                        <edge>from P6 to T4</edge>
                        <transition>T4: Inspect Hardware</transition>
                        <edge>from T4 to P7</edge>
                        <place>P7: Hardware Inspection Complete</place>
                        <edge>from P7 to T5</edge>
                        <transition>T5: Repair Hardware</transition>
                        <edge>from T5 to P8</edge>
                        <place>P8: Hardware Repair Complete</place>
                        <place>P9: Software Check</place>
                        <edge>from P9 to T6</edge>
                        <transition>T6: Inspect Software</transition>
                        <edge>from T6 to P10</edge>
                        <place>P10: Software Inspection Complete</place>
                        <edge>from P10 to T7</edge>
                        <transition>T7: Configure Software</transition>
                        <edge>from T7 to P11</edge>
                        <place>P11: Software Configuration Complete</place>
                        <convergence>
                            <edge>from P8 and P11 to T8</edge>
                            <transition>T8: Complete Parallel Activities</transition>
                            <edge>from T8 to P12</edge>
                            <place>P12: Parallel Activities Complete</place>
                        </convergence>
                    </correct>
                </example>
            </section>

            <section>
                <name>Loops and Cycles</name>
                <description>Repeating actions based on conditions.</description>
                <petrinetConstruct>Loop back to a previous place.</petrinetConstruct>
                <example>
                    <correct>
                        <loopName>Loop for Error Detection and Repair</loopName>
                        <place>P14: Detect Error</place>
                        <edge>from P14 to T12</edge>
                        <transition>T12: Repair Error</transition>
                        <edge>from T12 to P15</edge>
                        <place>P15: Test Functionality</place>
                        <edge>from P15 to T13 if error detected (loop)</edge>
                        <edge>from P15 to T14 if no error detected</edge>
                        <transition>T13: Error detected</transition>
                        <edge>from T13 to P14 (Detect Error)</edge>
                        <transition>T14: Test successful</transition>
                        <edge>from T14 to End Point (Test successful)</edge>
                    </correct>
                </example>
            </section>

            <section>
                <name>End Points</name>
                <description>Conclusion of the process.</description>
                <petrinetConstruct>Place with incoming edges but no outgoing edges.</petrinetConstruct>
                <example>
                    <correct>
                        <description>Completion of Repair</description>
                        <place>P9: Repair Complete</place>
                        <edge>from T8 (last transition) to P9 (Repair Complete)</edge>
                    </correct>
                </example>
            </section>
        </modelingInstructions>

        <output>
            <places>
                <place>P1: Customer Brings Computer</place>
                <place>P2: Defective Computer Received</place>
                <place>P3: Repair Cost Calculated</place>
                <place>P4: Accept Costs</place>
                <place>P5: Reject Costs</place>
                ...
            </places>
            <transitions>
                <transition>T1: Receive Computer</transition>
                <transition>T2: Check Defect</transition>
                <transition>T3: Calculate Repair Cost</transition>
                <transition>T4: Decide on Accepting Costs</transition>
                <transition>T5: Decide on Rejecting Costs</transition>
                ...
            </transitions>
            <edges>
                <edge>P1 --> T1</edge>
                <edge>T1 --> P2</edge>
                <edge>P2 --> T2</edge>
                <edge>T2 --> P3</edge>
                <edge>P3 --> T4</edge>
                <edge>P3 --> T5</edge>
                ...
            </edges>
        </output>

        <importantNotes>
            <note>Depend strictly on the texts content.</note>
        </importantNotes>
    </prompt>
    """
    
    # Extrahiere die strukturierte Prozessbeschreibung aus dem AgentState.
    process = state["process_description"]
    process_description = process[-1]
    
    # Erstelle die Eingabe für das LLM.
    message = [
    SystemMessage(content=modell_prompt),
    HumanMessage(content=process_description)]
    
    # Rufe das LLM auf und speichere das generierte Petri-Netz.
    response = llm.invoke(message).content
    
    # Speichere das Petri-Netz im AgentState unter 'process_modell'.
    return {"process_modell": [response]}

# Überprüft das generierte Petri-Netz auf Korrektheit.

# Diese Funktion verwendet ein LLM, um das generierte Petri-Netz-Modell auf 
# seine strukturelle Integrität und logische Konsistenz zu überprüfen. Es 
# werden detaillierte Rückmeldungen zu erkannten Fehlern oder Problemen generiert.

# Args:
    # state (AgentState): Der aktuelle Status des Agenten, enthält das generierte Petri-Netz-Modell. 

# Returns:
    # dict: Ein Dictionary, das die generierte Kritik am Petri-Netz im AgentState unter dem Schlüssel "critique" speichert.
def critique(state):
    
    # Definiere den Prompt für die Überprüfung des Petri-Netz.
    # Der Prompt enthält Anweisungen, auf welche Aspekte das LLM bei der Überprüfung 
    # achten soll und wie das Feedback zu strukturieren ist.
    critique_prompt = """
    You are an expert in process modeling and Petri Nets. Your task is to review the provided Petri Net model for correctness and provide detailed feedback on its accuracy. Follow these steps:
    **Verify Structural Integrity:**
    1. Check if there are edges that connect two places or two transitions with each other. That is the only thing that is not allowed in a petri net. Edges between the same type of object.
    **Check Process Flow:**
    1. Verify that the petri net starts with a Place and ends with a Place.
    **Provide Feedback:**
    1. Highlight any detected errors in the model.
    2. Confirm the correctness of the model if no errors are found.
    

    **Example Valid Output:**
    
    Correctness Feedback:
    The model starts with Place P1 and ends with Place P2.
    No invalid edges detected (Place to Place or Transition to Transition).
    The model is correct and does not require any changes.
    
    **Example Invalid Output with Feedback:**
    
    Errors Detected:
    Invalid Edge Detected:
    Edge from T1 to T2 (Error: Transition to Transition)
    Net Does Not Start with Place:
    First edge is from Transition T1 (Error: Net starts with Transition)
    Correct Invalid Edge:
    Change the edge from T1 to T2 to connect via a Place, e.g., P3.
    Corrected Edge: T1 --> P3, P3 --> T2
    Start Net with Place:
    Ensure the first edge starts from a Place, e.g., P1.
    Corrected Edge: P1 --> T1
    
    **Output:**
    Only answer with the issues and then the recommendation on how to solve the issues.
    """
    
    # Extrahiere das Petri-Netz aus dem AgentState.
    modell = state["process_modell"]
    process_modell = modell[-1]
    
    # Erstelle die Eingabe für das LLM.
    message = [
    SystemMessage(content=critique_prompt),
    HumanMessage(content=process_modell)]
    
    # Rufe das LLM auf und speichere die generierte Kritik.
    response = llm.invoke(message).content
    
    # Speichere die Kritik im AgentState unter 'critique'.
    return {"critique": [response]}

# Verbessert das Petri-Netz basierend auf der Kritik.
# Diese Funktion nutzt ein LLM, um das generierte Petri-Netz-Modell 
# basierend auf der zuvor generierten Kritik zu verbessern.  

# Args:
    # state (AgentState): Der aktuelle Status des Agenten, enthält das Petri-Netz-Modell und die Kritik. 

# Returns:
    # dict: Ein Dictionary, das das verbesserte Petri-Netz-Modell im AgentState unter dem Schlüssel "process_modell" speichert.
def improve(state):
    
    # Definiere den Prompt für die Verbesserung des Petri-Netz.
    # Der Prompt enthält Anweisungen, wie das LLM die Kritik zu interpretieren 
    # und das Petri-Netz zu korrigieren hat.
    improve_prompt = """
    You are an expert in process modeling and Petri Nets. Your task is to improve a provided Petri Net model based on the given feedback. You should only answer with the improved Petri Net, don't explain what you changed or anything else, just the Petri Net. Follow these steps to update the model accurately:

    Input:
    Petri Net Model:

    Places, Transitions, and Edges in the current model.
    Feedback:

    Detailed feedback highlighting errors and suggested improvements.
    Steps to Improve the Petri Net:
    Review Feedback:

    Thoroughly understand the feedback provided.
    Identify specific errors and the recommended changes.
    Modify the Petri Net:

    Correct invalid edges (e.g., Place to Place or Transition to Transition).
    Just answer with the improved Petri Net nothing more.
    
    <output>
            <places>List all places with their names.</places>
            <transitions>List all transitions with their names.</transitions>
            <edges>List all edges in the format "Place --> Transition" or "Transition --> Place".</edges>
            <summary>Do not include any summaries or explanations. Only provide the structured lists of places, transitions, and edges.</summary>
        </output>
    """
    
    # Extrahiere das Petri-Netz und die Kritik aus dem AgentState.
    modell = state["process_modell"]
    process_modell = modell[-1]
    critique = state["critique"]
    last_critique = critique[-1]
    
    # Wandle das Petri-Netz und die Kritik in Strings um.
    process_modell_str = str(process_modell)
    last_critique_str = str(last_critique)
    
    # Kombiniere das Petri-Netz und die Kritik in einer einzigen Zeichenkette.
    process_modell_and_critique = "The process modell: " + process_modell_str + "and the critique: " + last_critique_str
    
    # Erstelle die Eingabe für das LLM.
    message = [
    SystemMessage(content=improve_prompt),
    HumanMessage(content=process_modell_and_critique)]
    
    # Rufe das LLM auf und speichere das verbesserte Petri-Netz.
    response = llm.invoke(message).content
    
    # Speichere das verbesserte Petri-Netz im AgentState.
    return {"process_modell": [response]}

In [7]:
# Definiere Pydantic Klassen für das Schema des Petri-Netzes. Hier wird das Schema des Horus Business Modeler
# definiert.
# Diese Klassen werden verwendet, um die Ausgabe des LLM zu strukturieren 
# und in ein JSON-Format zu überführen.
class Name(BaseModel):
    de: str = Field(description="The name in German")
    en: str = Field(description="The name in English")
    
class Element(BaseModel):
    name: Name = Field(description="The name of the element in different languages")
    self: int = Field(description="The unique identifier of the element")
    type: str = Field(description="The type of the element", enum=["Activity", "Object Store"])

class Edge(BaseModel):
    from_: int = Field(alias="from", description="The ID of the source element")
    to: int = Field(description="The ID of the target element")

class PetriNetTransition(BaseModel):
    elements: List[Element] = Field(description="Elements in the Petri net")
    edges: List[Edge] = Field(description="Edges connecting elements in the Petri net")

# Parst die Ausgabe des LLM und generiert ein Petri-Netz im JSON-Format.
# Diese Funktion extrahiert das vom LLM generierte Petri-Netz aus dem AgentState, 
# formatiert es mithilfe eines Prompts und parst es mit dem JSON Output-Parser 
# in ein strukturiertes JSON-Format. Das Ergebnis wird in einer JSON-Datei gespeichert.

# Args:
    # state (AgentState): Der aktuelle Status des Agenten, enthält das generierte Petri-Netz-Modell.

# Returns:
    # PetriNetTransition: Das geparste Petri-Netz-Modell als Pydantic-Objekt.
def call_pydantic_output_parser(state):
    
    # Extrahiere das Petri-Netz aus dem 'AgentState'.
    modell = state["process_modell"]
    process_modell = modell[-1]
    
    # Definiere den Prompt für den Output-Parser.
    # Der Prompt enthält Anweisungen, wie die Ausgabe des LLM 
    # zu formatieren ist, um sie mit dem JSON Output-Parser verarbeiten zu können.
    prompt = ChatPromptTemplate.from_messages([
        ("system", """
        You are an expert in transforming an input into a desired output.
        You only use the information provided named Transitions, Places and Edges to create a Petri net.
        Label Places as 'Object Store'.
        Label Transitions as 'Activity'.
        Don't ever create an edge between an 'Activity' and an 'Activity'.
        Don't ever create an edge between an 'Object Store' and an 'Object Store'.
        Edges are only allowed between an 'Object Store' and an 'Activity'.
        
        {format_instructions}

        Make sure to answer in the correct format
        """),
        ("human", "{input}")
    ])

    # Initialisiere den JSON Output-Parser mit dem Petri-Netz-Schema.
    parser = JsonOutputParser(pydantic_object=PetriNetTransition)

    # Erstelle eine Verarbeitungskette, die den Prompt, das LLM und den Parser verbindet.
    chain = prompt | llm | parser
    
    # Rufe die Verarbeitungskette mit dem Petri-Netz als Eingabe auf.
    result = chain.invoke({
        "input": process_modell,
        "format_instructions": parser.get_format_instructions()
    })
    
    # Speichere das Ergebnis als JSON-Datei.
    with open('process_description.json', 'w') as json_file:
        json.dump(result, json_file, indent=4, ensure_ascii=False)

    print("Die JSON-Datei wurde erfolgreich erstellt und gespeichert.")
    
    # Gib das geparste Petri-Netz zurück.
    return result

In [9]:
# Definiere den Workflow-Graphen.
# Der Workflow-Graph beschreibt die Abfolge der einzelnen Schritte im Prozess der 
# Prozessmodellierung mithilfe des Sprachmodells.
workflow = StateGraph(AgentState)

# Definiere die Knoten im Workflow-Graphen.
# Jeder Knoten repräsentiert einen Schritt im Workflow und ist mit einer Funktion verknüpft, 
# die wenn der Workflow den Knoten erreicht, ausgeführt wird.
workflow.add_node("Questions", generateQuestions)
workflow.add_node("Answers", ask_human)
workflow.add_node("Integrate", incorporate)
workflow.add_node("Structure", structure)
workflow.add_node("Model", modell)
workflow.add_node("Critic", critique)
workflow.add_node("Improve", improve)
workflow.add_node("OutputParser", call_pydantic_output_parser)

# Definiere die Kanten zwischen den Knoten im Workflow-Graphen.
# Die Kanten definieren die Reihenfolge, in der die Knoten abgearbeitet werden.
workflow.add_edge('Questions', 'Answers')
workflow.add_edge('Answers', 'Integrate')
workflow.add_edge('Integrate', 'Structure')
workflow.add_edge('Structure', 'Model')
workflow.add_edge('Model', 'Critic')
workflow.add_edge('Critic', 'Improve')
workflow.add_edge('Improve', 'OutputParser')

# Kompiliere den Workflow-Graphen.
workflow.set_entry_point("Questions")
workflow.set_finish_point("OutputParser")

graph = workflow.compile()

In [ ]:
# Beispielprozessbeschreibung.
# Diese Prozessbeschreibung dient als Eingabe für den Workflow.
process_description = """
When an enquiry is received by the customer office, it is immediately recorded in the system as a new case. An employee then liaises with the customer to obtain the necessary additional information. At the same time, another employee begins to collate the necessary documents and check that they are complete. As soon as both activities have been completed, the case is closed.
"""
user_input = {"process_description": [process_description]}

# Führe den Workflow aus und gib die Ausgaben der einzelnen Knoten aus.
for output in graph.stream(user_input):
    # Die stream()-Methode liefert ein Dictionary mit den Ausgaben der einzelnen Knoten zurück.
    for key, value in output.items():
        print(f"Output from node '{key}':")
        print("---")
        print(value)
    print("\n---\n")